In [5]:
# !pip install xgboost

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

In [7]:
df = pd.read_csv('final_data.csv')
df.drop(columns = ['player', 'team', 'name'], inplace = True)
df.dropna(inplace = True)
df.shape

(10754, 19)

In [8]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X = df.drop('position_encoded', axis=1)
y = df['position_encoded']

y_encoded = le.fit_transform(y)

df['position_encoded'] = y_encoded


In [9]:
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

In [10]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [11]:
logistic_model = LogisticRegression(multi_class='multinomial', solver='saga')

logistic_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', logistic_model)
])

logistic_param_grid = {
    'classifier__penalty': ['l1', 'l2'],
    'classifier__C': [0.1, 1.0, 10.0]
}

logistic_grid_search = GridSearchCV(logistic_pipeline, logistic_param_grid, cv=5, n_jobs= -1)

In [12]:
xgb_model = xgb.XGBClassifier(objective='multi:softprob')

xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', xgb_model)
])

xgb_param_grid = {
    'classifier__learning_rate': [0.01, 0.1, 0.2],
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [3, 5, 7]
}

xgb_grid_search = GridSearchCV(xgb_pipeline, xgb_param_grid, cv=5, n_jobs= -1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

logistic_grid_search.fit(X_train, y_train)
print("Best parameters for Logistic Regression:", logistic_grid_search.best_params_)

xgb_grid_search.fit(X_train, y_train)
print("Best parameters for XGBoost:", xgb_grid_search.best_params_)

c:\Users\osama\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best parameters for Logistic Regression: {'classifier__C': 10.0, 'classifier__penalty': 'l1'}


ValueError: 
All the 135 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\osama\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\osama\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\osama\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\pipeline.py", line 475, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "c:\Users\osama\AppData\Local\Pro

In [ ]:
from sklearn.metrics import classification_report
y_pred_logistic = logistic_grid_search.predict(X_test)

# Print classification report
print("Classification Report for Logistic Regression:")
print(classification_report(y_test, y_pred_logistic))

Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       246
           1       1.00      0.99      0.99       706
           2       0.99      1.00      0.99       619
           3       1.00      1.00      1.00       580

    accuracy                           0.99      2151
   macro avg       1.00      1.00      1.00      2151
weighted avg       0.99      0.99      0.99      2151



In [ ]:
y_pred_xgb = xgb_grid_search.predict(X_test)

# Print classification report
print("Classification Report for XGBoost:")
print(classification_report(y_test, y_pred_xgb))

Classification Report for XGBoost:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       246
           1       1.00      1.00      1.00       706
           2       1.00      1.00      1.00       619
           3       1.00      1.00      1.00       580

    accuracy                           1.00      2151
   macro avg       1.00      1.00      1.00      2151
weighted avg       1.00      1.00      1.00      2151

